<h1>Google Landmark Recognition Project<h1>

In [1]:
#importing neccessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras import datasets, layers, models
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from sklearn.utils import class_weight

<h2>1. Exploratory Analysis<h2>

In [2]:
#importing the dataset with classes and id
classes = pd.read_csv("/Users/seshu/Desktop/CPSC/train.csv")

In [3]:
#dataset head
classes.head()

,id,landmark_id
0,17660ef415d37059,1
1,92b6290d571448f6,1
2,cd41bf948edc0340,1
3,fb09f1e98c6d2f70,1
4,25c9dfc7ea69838d,7


In [4]:
#dimensions of the dataset
classes.shape

(1580470, 2)

In [5]:
#number of classes/landmarks
len(pd.unique(classes['landmark_id']))

81313

In [6]:
#label with maximum images
classes['landmark_id'].value_counts().idxmax()

138982

<h2>2. Pre-processing and exploratory analysis<h2>

In [7]:
#grouping classes and printing corresponding image count
classes_cnt = classes['landmark_id'].value_counts().sort_values(ascending=False)
classes_df = pd.DataFrame({'Label id':classes_cnt.index, 'Images':classes_cnt.values})

In [8]:
#viewing the dataset
classes_df

,Label id,Images
0,138982,6272
1,126637,2231
2,20409,1758
3,83144,1741
4,113209,1135
...,...,...
81308,72967,2
81309,143694,2
81310,24793,2
81311,143896,2


In [9]:
#summary of the dataset
classes_df.describe()

,Label id,Images
count,81313.000000,81313.000000
mean,101291.902070,19.436867
std,58740.916419,42.877489
min,1.000000,2.000000
25%,50237.000000,5.000000
50%,101073.000000,9.000000
75%,152146.000000,20.000000
max,203092.000000,6272.000000


In [10]:
#Most common image count among the classes
classes_df['Images'].value_counts().idxmax()

4

In [11]:
#max count of images in a class
classes_df['Images'].max()

6272

In [12]:
#number of unique values in images column
len(pd.unique(classes_df['Images']))

430

In [13]:
#number of classes with count of images equal to 4
len(classes_df[classes_df['Images']==4])

8306

In [18]:
#count of images with images greater than 30
classes_df[classes_df['Images']<30]['Images'].sum()

659749

In [17]:
#classes with images greater than 30
classes_df[classes_df['Images']>30]['Label id'].count()

12480

<h2>3. Undersampling the dataset<h2>

In [19]:
#labels of classes with images greater than 30
df_thirty = classes_df[classes_df['Images']>30]['Label id']
df_thirty = pd.DataFrame(df_thirty)

In [49]:
#viewing the dataset
df_thirty

,Label id
0,138982
1,126637
2,20409
3,83144
4,113209
...,...
12475,159925
12476,159814
12477,116213
12478,47471


In [53]:
#undersampling the data
def under_sample():
    
    #this function returns the undersampled dataset. It selects all the classes with image count greater than 30
    #and undersamples by random selection of 30.
    
    classes2 = classes
    
    for i in range(0, len(df_thirty)):
        
        label_id = df_thirty.iloc[i, 0]
        subset = classes2.loc[classes2['landmark_id'] == label_id]
        sample = subset.sample(n = 30, random_state = 918)
        classes2.drop(classes2[classes2['landmark_id'] == label_id].index, inplace = True)
        data = [classes2, sample]
        classes2 = pd.concat(data)
        
    return classes2

In [ ]:
#calling the function under_sample()
classes2 = under_sample()

<h2>4. Dropping classes with count of images less than 3 <h2>

In [21]:
#labels of classes with images less than 3
df_two = classes_df[classes_df['Images']<3]['Label id']
df_two = pd.DataFrame(df_two)

In [26]:
#viewing the dataset
df_two

,Label id
76563,90893
76564,128632
76565,90932
76566,161633
76567,38386
...,...
81308,72967
81309,143694
81310,24793
81311,143896


In [28]:
#Removing classes with image count less than 2
def drop_two():
    
    #this function returns the dataset after dropping all the classes with images less than 3
    
    for i in range(0, len(df_two)):
        
        label_id = df_two.iloc[i, 0]
        classes2.drop(classes2[classes2['landmark_id'] == label_id].index, inplace = True)

    return classes2

In [ ]:
#calling the function drop_two()
under_drop = drop_two()

In [33]:
#saved the dataset under_drop to local machine

In [34]:
#importing the dataset under_drop from local machine
under_drop = pd.read_csv("/Users/seshu/Desktop/Assignment CPSC/under_drop.csv")

In [35]:
under_drop

,Unnamed: 0,id,landmark_id
0,0,17660ef415d37059,1
1,1,92b6290d571448f6,1
2,2,cd41bf948edc0340,1
3,3,fb09f1e98c6d2f70,1
4,4,25c9dfc7ea69838d,7
...,...,...,...
1040784,547544,ad4dba98d38d2461,69852
1040785,547524,06d19b43d65a01b9,69852
1040786,547529,3d5f3c284aa5d2f9,69852
1040787,547548,c3c96ed6603f673d,69852


In [36]:
#dropping 'Unnamed: 0' column
under_drop = under_drop.drop('Unnamed: 0', axis=1)

In [37]:
#viewing the dataset
under_drop

,id,landmark_id
0,17660ef415d37059,1
1,92b6290d571448f6,1
2,cd41bf948edc0340,1
3,fb09f1e98c6d2f70,1
4,25c9dfc7ea69838d,7
...,...,...
1040784,ad4dba98d38d2461,69852
1040785,06d19b43d65a01b9,69852
1040786,3d5f3c284aa5d2f9,69852
1040787,c3c96ed6603f673d,69852


In [38]:
#saving the labels in y_train to create weights
y_train = under_drop[["landmark_id"]]

<h2>5. Preprocessing and loading training images <h2>

In [43]:
#preprocessing the dataset under_drop before loading images

#saving the common path in a variable PATH
PATH = '/Volumes/T7 Shield/CPSC Google landmark data'

#adding a new column named path which saves path for each image
under_drop['path'] = PATH + "/" + classes.id.str[0] + "/" + classes.id.str[1] + "/" + classes.id.str[2] + "/" + classes.id + ".jpg"
under_drop

,id,landmark_id,path
0,17660ef415d37059,1,/Volumes/T7 Shield/CPSC Google landmark data/1...
1,92b6290d571448f6,1,/Volumes/T7 Shield/CPSC Google landmark data/9...
2,cd41bf948edc0340,1,/Volumes/T7 Shield/CPSC Google landmark data/c...
3,fb09f1e98c6d2f70,1,/Volumes/T7 Shield/CPSC Google landmark data/f...
4,25c9dfc7ea69838d,7,/Volumes/T7 Shield/CPSC Google landmark data/2...
...,...,...,...
1040784,ad4dba98d38d2461,69852,/Volumes/T7 Shield/CPSC Google landmark data/b...
1040785,06d19b43d65a01b9,69852,/Volumes/T7 Shield/CPSC Google landmark data/d...
1040786,3d5f3c284aa5d2f9,69852,/Volumes/T7 Shield/CPSC Google landmark data/e...
1040787,c3c96ed6603f673d,69852,/Volumes/T7 Shield/CPSC Google landmark data/e...


In [45]:
#ensuring labels are of string type
under_drop['landmark_id'] = under_drop['landmark_id'].astype(str)
under_drop

,id,landmark_id,path
0,17660ef415d37059,1,/Volumes/T7 Shield/CPSC Google landmark data/1...
1,92b6290d571448f6,1,/Volumes/T7 Shield/CPSC Google landmark data/9...
2,cd41bf948edc0340,1,/Volumes/T7 Shield/CPSC Google landmark data/c...
3,fb09f1e98c6d2f70,1,/Volumes/T7 Shield/CPSC Google landmark data/f...
4,25c9dfc7ea69838d,7,/Volumes/T7 Shield/CPSC Google landmark data/2...
...,...,...,...
1040784,ad4dba98d38d2461,69852,/Volumes/T7 Shield/CPSC Google landmark data/b...
1040785,06d19b43d65a01b9,69852,/Volumes/T7 Shield/CPSC Google landmark data/d...
1040786,3d5f3c284aa5d2f9,69852,/Volumes/T7 Shield/CPSC Google landmark data/e...
1040787,c3c96ed6603f673d,69852,/Volumes/T7 Shield/CPSC Google landmark data/e...


In [177]:
batch_size = 32
target_shape=(192,192)

datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = datagen.flow_from_dataframe(
        dataframe = under_drop,
        x_col = "path",
        y_col = "label",
        target_size = target_shape,
        class_mode = "sparse",
        batch_size = batch_size
)

Found 1040789 validated image filenames belonging to 76563 classes.


<h2>6. Computing weightages for the classes <h2>

In [ ]:
#generating class weights based on training data
counter = Counter(train_generator.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}  

<h2> 7. Defining function for Image Augmentation <h2>  

In [23]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(192, 
                                                              192,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)


2022-08-11 00:08:10.217484: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<h2>Approach 1 : model building<h2>

<h2>8. Creating a CNN model from basics and training on the training data. <h2>

In [31]:
model = Sequential()

model.add(data_augmentation)
model.add(Conv2D(16, kernel_size=(3, 3), padding = 'same', activation = 'LeakyReLU', input_shape = (192, 192, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size=(3, 3), padding = 'same', activation = 'LeakyReLU'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), padding = 'same', activation = 'LeakyReLU'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.1))

model.add(Flatten())

model.add(Dense(128, activation='LeakyReLU'))

model.add(Dense(256, activation='LeakyReLU'))

model.add(Dropout(0.1))

model.add(Dense(76563 , activation='softmax'))
    
    
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 192, 192, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 192, 192, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 96, 96, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 96, 96, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 48, 48, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 48, 48, 64)       

In [ ]:
#setting the hyperparameters for the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['categorical_accuracy'])

In [153]:
#fitting the model
model.fit(train_generator, 
            epochs=1, 
            steps_per_epoch=train_generator.samples//batch_size, 
            validation_data = validation_generator,
            validation_steps=validation_generator.samples//batch_size)
            class_weight = class_weights
            )

29272/29272 [==============================] - 47207s 2s/step - loss: 11.1988 - Accuracy: 5.5515e-05 - val_loss: 17.7632 - val_Accuracy: 0.0000e+00


<h2>Approach 2 : Using pre trained model<h2>

<h2>9. Loading training data and validation data<h2>

In [19]:
#loading the data using ImageDataGenerator
batch_size = 32
target_shape=(192,192)

datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1,
    fill_mode='nearest')

train_generator = datagen.flow_from_dataframe(
        dataframe = under_drop,
        x_col = "path",
        y_col = "label",
        target_size = target_shape,
        color_mode = "rgb",
        class_mode = "sparse",
        batch_size = batch_size,
        subset = 'training'
)

validation_generator = datagen.flow_from_dataframe(
        dataframe = under_drop,
        x_col = "path",
        y_col = "label",
        target_size = target_shape,
        color_mode = "rgb",
        class_mode = "sparse",
        batch_size = batch_size,
        subset = 'validation'
)

Found 936711 validated image filenames belonging to 76563 classes.
Found 104078 validated image filenames belonging to 76563 classes.


<h2>10. Training on resentv2 model<h2>

In [24]:
#training using renet50v2 model
resnet = Sequential()

pre_model = tf.keras.applications.ResNet50V2(
    include_top = False,
    input_shape = (192, 192, 3),
    pooling = 'max',
    weights = 'imagenet',
    classes= 76563)

for layer in pre_model.layers:
    layer.trainable = False

resnet.add(data_augmentation)
resnet.add(pre_model)
resnet.add(Flatten())
resnet.add(Dense(256, activation='LeakyReLU'))
resnet.add(Dense(512, activation='LeakyReLU'))
resnet.add(Dense(76563 , activation='softmax'))

94668760/94668760 [==============================] - 9s 0us/step


In [28]:
#summary of the model
resnet.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 192, 192, 3)       0         
                                                                 
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dense_1 (Dense)             (None, 512)               131584    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 76563)            

In [29]:
#setting hyperparameters
resnet.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['Accuracy'])

In [31]:
#fitting the model on the data
resnet.fit(train_generator, 
            epochs=15, 
            steps_per_epoch= 1500, 
            validation_data = validation_generator,
            validation_steps= 300
            )

Epoch 1/15
1500/1500 [==============================] - 2126s 1s/step - loss: 11.7066 - Accuracy: 1.2500e-04 - val_loss: 12.1300 - val_Accuracy: 0.0000e+00
Epoch 2/15
1500/1500 [==============================] - 2099s 1s/step - loss: 11.6830 - Accuracy: 3.3333e-04 - val_loss: 11.9237 - val_Accuracy: 0.0000e+00
Epoch 3/15
1500/1500 [==============================] - 2145s 1s/step - loss: 11.5805 - Accuracy: 7.5000e-04 - val_loss: 13.9055 - val_Accuracy: 0.0000e+00
Epoch 4/15
1500/1500 [==============================] - 2161s 1s/step - loss: 11.5243 - Accuracy: 0.0010 - val_loss: 14.1211 - val_Accuracy: 0.0000e+00
Epoch 5/15
1500/1500 [==============================] - 2154s 1s/step - loss: 11.4913 - Accuracy: 0.0010 - val_loss: 14.8384 - val_Accuracy: 0.0000e+00
Epoch 6/15
1500/1500 [==============================] - 2157s 1s/step - loss: 11.4846 - Accuracy: 0.0013 - val_loss: 15.3554 - val_Accuracy: 0.0000e+00
Epoch 7/15
1500/1500 [==============================] - 2162s 1s/step - loss